# terminal execution

In [47]:
! pip3 install pyarrow

     |████████████████████████████████| 17.8 MB 655 kB/s eta 0:00:01    |███████▎                        | 4.0 MB 4.5 MB/s eta 0:00:04


# user input

In [1]:
basePath = '/home/bobby/'
codePath = 'ml/src'
dataPath = 'dat'

projectName = 'melbourne_housing_snapshot'
rawInputPath = 'melb_data.csv'

# dependencies

In [2]:
import os
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

import datetime

# initialization

In [3]:
### Initialization
BASE_DIRECTORY = os.path.dirname(basePath)
CODE_DIRECTORY = os.path.join(BASE_DIRECTORY, codePath)
DATA_DIRECTORY = os.path.join(BASE_DIRECTORY, dataPath)

projectDir = os.path.join(DATA_DIRECTORY, projectName)
inDir = os.path.join(projectDir, 'in')
outDir = os.path.join(projectDir, 'out')
rawInput = os.path.join(inDir, rawInputPath)

y_eda_outfile = os.path.join(outDir, 'y_eda.feather')
y_preprocessed_outfile = os.path.join(outDir, 'y_preprocessed.feather')
y_featureengineered_outfile = os.path.join(outDir, 'y_featureengineered.feather')
y_modelled_outfile = os.path.join(outDir, 'y_modelled.feather')
y_resulted_outfile = os.path.join(outDir, 'y_resulted.feather')
y_test_outfile = os.path.join(outDir, 'y_test.feather')


x_eda_outfile = os.path.join(outDir, 'x_eda.feather')
x_preprocessed_outfile = os.path.join(outDir, 'x_preprocessed.feather')
x_featureengineered_outfile = os.path.join(outDir, 'x_featureengineered.feather')
x_modelled_outfile = os.path.join(outDir, 'x_modelled.feather')
x_resulted_outfile = os.path.join(outDir, 'x_resulted.feather')

x_test_outfile = os.path.join(outDir, 'x_test.feather')


# custom classes and functions

In [4]:
### Custom classes and functions


# data input

In [5]:
rawDataframe = pd.read_csv(rawInput)

x = rawDataframe.drop('Price', axis=1)
y = rawDataframe.Price

train_x, test_x, train_y, test_y = train_test_split(x, y, random_state = 0)
test_x.reset_index().to_feather(x_test_outfile)
test_y.to_frame().reset_index().to_feather(y_test_outfile)

# exploratory data analysis

In [7]:
rawDataframe.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [8]:
rawDataframe.describe()

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
count,13580.000000,1.358000e+04,13580.000000,13580.000000,13580.000000,13580.000000,13518.000000,13580.000000,7130.000000,8205.000000,13580.000000,13580.000000,13580.000000
mean,2.937997,1.075684e+06,10.137776,3105.301915,2.914728,1.534242,1.610075,558.416127,151.967650,1964.684217,-37.809203,144.995216,7454.417378
std,0.955748,6.393107e+05,5.868725,90.676964,0.965921,0.691712,0.962634,3990.669241,541.014538,37.273762,0.079260,0.103916,4378.581772
min,1.000000,8.500000e+04,0.000000,3000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1196.000000,-38.182550,144.431810,249.000000
25%,2.000000,6.500000e+05,6.100000,3044.000000,2.000000,1.000000,1.000000,177.000000,93.000000,1940.000000,-37.856822,144.929600,4380.000000
50%,3.000000,9.030000e+05,9.200000,3084.000000,3.000000,1.000000,2.000000,440.000000,126.000000,1970.000000,-37.802355,145.000100,6555.000000
75%,3.000000,1.330000e+06,13.000000,3148.000000,3.000000,2.000000,2.000000,651.000000,174.000000,1999.000000,-37.756400,145.058305,10331.000000
max,10.000000,9.000000e+06,48.100000,3977.000000,20.000000,8.000000,10.000000,433014.000000,44515.000000,2018.000000,-37.408530,145.526350,21650.000000


In [9]:
y_eda = rawDataframe.Price
x_eda = rawDataframe.drop('Price', axis=1)
y_eda.to_frame().to_feather(y_eda_outfile)
x_eda.to_feather(x_eda_outfile)

# pre-processing

In [10]:
y_preprocessed = pd.read_feather(y_eda_outfile)
x_preprocessed = pd.read_feather(x_eda_outfile)

In [11]:
y_preprocessed.to_feather(y_preprocessed_outfile)
x_preprocessed.to_feather(x_preprocessed_outfile)

# feature engineering

In [12]:
y_featureengineered = pd.read_feather(y_preprocessed_outfile)
x_featureengineered = pd.read_feather(x_preprocessed_outfile)

In [13]:
y_featureengineered.to_feather(y_featureengineered_outfile)
x_featureengineered.to_feather(x_featureengineered_outfile)

# modeling

In [14]:
y_modelled = pd.read_feather(y_featureengineered_outfile)
x_modelled = pd.read_feather(x_featureengineered_outfile)

In [15]:
dtr = DecisionTreeRegressor(random_state=1)

dtr.fit(x, y)

ValueError: could not convert string to float: 'Abbotsford'

# validation

In [ ]:
### Testing


# comparison with others

# double check

# results

In [ ]:
### Results
